![](./graphs/dl_banner.jpg)

# 用tensorflow low level API构建循环神经网络
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

## 0.知识背景：循环神经网络
循环神经网络或RNN是一类用于处理序列数据的神经网络，典型的时间序列数据场景是文本，RNN在自然语言处理中发挥着很大的作用。循环神经网络会对序列的每个时间点数据处理，典型结构如下。
![RNN](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

循环神经网络中最常用到的两个变种结构为LSTM和GRU，它们在对数据计算的cell结构上做了一些调整，分别如下：
![LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)
![GRU](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png)

## 0.问题背景：MNIST 手写数字
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

这是一个非常经典的问题，我们的对象是手写数字图片，我们需要根据手写数字图片的输入，构建线性的分类器(softmax分类器)去区分图片是手写数字0-9中的哪一个。这个问题的数据集，每一张图片的表示是长宽为28的矩阵，我们有时候会把它展开成784维的向量。MNIST手写数据集长成下面这样。

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

更多的信息可以参考: http://yann.lecun.com/exdb/mnist/

## 1.引入工具库

In [1]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import numpy as np
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config, ...)

## 2.设定超参数
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [2]:
# 训练超参数
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 10

# 神经网络超参数
num_input = 28 # MNIST数据输入(数据形状: 28*28)，一个时间点上输入28个值
timesteps = 28 # 总共会有28个timestep
num_hidden = 128 # 神经元个数
num_classes = 10 # MNIST总共类别(0-9总共10个手写数字)


# 占位符
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# 变量
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

## 3.准备数据

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## 4.构建模型
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [4]:
def RNN(x, weights, biases):

    # RNN是时间序列型网络结构，我们需要按照它的形式去组织数据
    # 我们把数据组织成这样的形式: (batch_size, timesteps, n_input)
    # 也就是说一批样本batch_size个样本，总共会以timesteps个长度为n_input的向量形式送入

    # 为了得到timesteps个(batch_size, n_input)形态的输入
    x = tf.unstack(x, timesteps, 1)

    # 定义RNN中单个cell的格式
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # 用cell串成LSTM网络
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # 结果要通过全连接层
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

## 5.计算损失与优化
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [5]:
# 构建RNN网络进行预估
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# 定义损失计算方法与优化器
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 评估模型
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初始化变量
init = tf.global_variables_initializer()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## 6.在session当中完成计算图计算(损失计算与优化、参数更新迭代)
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [6]:
# 在session当中开始训练
with tf.Session() as sess:

    # 初始化
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # 优化
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # 计算准确率
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("第" + str(step) + "步, 一个minibatch上的损失为 " + \
                  "{:.4f}".format(loss) + ", 训练准确率为 " + \
                  "{:.3f}".format(acc))

    print("完成训练！")

    # 计算准确率
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("测试准确率为 ", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

第1步, 一个minibatch上的损失为 2.8395, 训练准确率为 0.125
第10步, 一个minibatch上的损失为 2.6570, 训练准确率为 0.094
第20步, 一个minibatch上的损失为 2.5128, 训练准确率为 0.141
第30步, 一个minibatch上的损失为 2.4960, 训练准确率为 0.102
第40步, 一个minibatch上的损失为 2.3838, 训练准确率为 0.156
第50步, 一个minibatch上的损失为 2.3620, 训练准确率为 0.102
第60步, 一个minibatch上的损失为 2.3865, 训练准确率为 0.148
第70步, 一个minibatch上的损失为 2.3292, 训练准确率为 0.156
第80步, 一个minibatch上的损失为 2.3350, 训练准确率为 0.094
第90步, 一个minibatch上的损失为 2.2571, 训练准确率为 0.219
第100步, 一个minibatch上的损失为 2.1770, 训练准确率为 0.250
第110步, 一个minibatch上的损失为 2.2455, 训练准确率为 0.203
第120步, 一个minibatch上的损失为 2.2253, 训练准确率为 0.242
第130步, 一个minibatch上的损失为 2.2330, 训练准确率为 0.234
第140步, 一个minibatch上的损失为 2.1929, 训练准确率为 0.266
第150步, 一个minibatch上的损失为 2.1402, 训练准确率为 0.305
第160步, 一个minibatch上的损失为 2.1811, 训练准确率为 0.289
第170步, 一个minibatch上的损失为 2.1618, 训练准确率为 0.281
第180步, 一个minibatch上的损失为 2.1668, 训练准确率为 0.305
第190步, 一个minibatch上的损失为 2.1707, 训练准确率为 0.281
第200步, 一个minibatch上的损失为 2.1308, 训练准确率为 0.242
第210步, 一个minibatch上的损失为 2.1381, 训练准确率为 0.258
第220步, 一个minibatch上的损

第1810步, 一个minibatch上的损失为 1.4496, 训练准确率为 0.531
第1820步, 一个minibatch上的损失为 1.4374, 训练准确率为 0.602
第1830步, 一个minibatch上的损失为 1.2054, 训练准确率为 0.680
第1840步, 一个minibatch上的损失为 1.3708, 训练准确率为 0.602
第1850步, 一个minibatch上的损失为 1.2297, 训练准确率为 0.641
第1860步, 一个minibatch上的损失为 1.2280, 训练准确率为 0.609
第1870步, 一个minibatch上的损失为 1.2361, 训练准确率为 0.594
第1880步, 一个minibatch上的损失为 1.2482, 训练准确率为 0.625
第1890步, 一个minibatch上的损失为 1.2675, 训练准确率为 0.609
第1900步, 一个minibatch上的损失为 1.2707, 训练准确率为 0.609
第1910步, 一个minibatch上的损失为 1.4465, 训练准确率为 0.586
第1920步, 一个minibatch上的损失为 1.3520, 训练准确率为 0.578
第1930步, 一个minibatch上的损失为 1.3033, 训练准确率为 0.648
第1940步, 一个minibatch上的损失为 1.2023, 训练准确率为 0.625
第1950步, 一个minibatch上的损失为 1.4681, 训练准确率为 0.492
第1960步, 一个minibatch上的损失为 1.1537, 训练准确率为 0.672
第1970步, 一个minibatch上的损失为 1.2704, 训练准确率为 0.602
第1980步, 一个minibatch上的损失为 1.3127, 训练准确率为 0.578
第1990步, 一个minibatch上的损失为 1.1610, 训练准确率为 0.672
第2000步, 一个minibatch上的损失为 1.2517, 训练准确率为 0.625
第2010步, 一个minibatch上的损失为 1.2492, 训练准确率为 0.555
第2020步, 一个minibatch上的损失为 1.2457, 训

第3600步, 一个minibatch上的损失为 0.7849, 训练准确率为 0.766
第3610步, 一个minibatch上的损失为 0.8150, 训练准确率为 0.789
第3620步, 一个minibatch上的损失为 1.0367, 训练准确率为 0.672
第3630步, 一个minibatch上的损失为 1.0403, 训练准确率为 0.680
第3640步, 一个minibatch上的损失为 0.9165, 训练准确率为 0.750
第3650步, 一个minibatch上的损失为 0.9427, 训练准确率为 0.742
第3660步, 一个minibatch上的损失为 0.8864, 训练准确率为 0.703
第3670步, 一个minibatch上的损失为 1.0981, 训练准确率为 0.695
第3680步, 一个minibatch上的损失为 0.8808, 训练准确率为 0.742
第3690步, 一个minibatch上的损失为 1.0115, 训练准确率为 0.695
第3700步, 一个minibatch上的损失为 1.0922, 训练准确率为 0.648
第3710步, 一个minibatch上的损失为 0.7560, 训练准确率为 0.773
第3720步, 一个minibatch上的损失为 0.9465, 训练准确率为 0.695
第3730步, 一个minibatch上的损失为 0.8064, 训练准确率为 0.742
第3740步, 一个minibatch上的损失为 0.8636, 训练准确率为 0.711
第3750步, 一个minibatch上的损失为 0.7932, 训练准确率为 0.727
第3760步, 一个minibatch上的损失为 0.9969, 训练准确率为 0.688
第3770步, 一个minibatch上的损失为 1.0657, 训练准确率为 0.633
第3780步, 一个minibatch上的损失为 0.8719, 训练准确率为 0.734
第3790步, 一个minibatch上的损失为 1.0350, 训练准确率为 0.680
第3800步, 一个minibatch上的损失为 1.0176, 训练准确率为 0.680
第3810步, 一个minibatch上的损失为 0.9702, 训

第5390步, 一个minibatch上的损失为 0.6744, 训练准确率为 0.805
第5400步, 一个minibatch上的损失为 0.6774, 训练准确率为 0.773
第5410步, 一个minibatch上的损失为 0.7595, 训练准确率为 0.750
第5420步, 一个minibatch上的损失为 0.7116, 训练准确率为 0.773
第5430步, 一个minibatch上的损失为 0.6402, 训练准确率为 0.797
第5440步, 一个minibatch上的损失为 0.7974, 训练准确率为 0.766
第5450步, 一个minibatch上的损失为 0.6152, 训练准确率为 0.781
第5460步, 一个minibatch上的损失为 0.6873, 训练准确率为 0.750
第5470步, 一个minibatch上的损失为 0.6918, 训练准确率为 0.805
第5480步, 一个minibatch上的损失为 0.7378, 训练准确率为 0.773
第5490步, 一个minibatch上的损失为 0.8968, 训练准确率为 0.734
第5500步, 一个minibatch上的损失为 0.7347, 训练准确率为 0.812
第5510步, 一个minibatch上的损失为 0.6131, 训练准确率为 0.805
第5520步, 一个minibatch上的损失为 0.6843, 训练准确率为 0.789
第5530步, 一个minibatch上的损失为 0.5782, 训练准确率为 0.828
第5540步, 一个minibatch上的损失为 0.8466, 训练准确率为 0.703
第5550步, 一个minibatch上的损失为 0.7197, 训练准确率为 0.820
第5560步, 一个minibatch上的损失为 0.7496, 训练准确率为 0.766
第5570步, 一个minibatch上的损失为 0.5346, 训练准确率为 0.844
第5580步, 一个minibatch上的损失为 0.5796, 训练准确率为 0.836
第5590步, 一个minibatch上的损失为 0.7240, 训练准确率为 0.750
第5600步, 一个minibatch上的损失为 0.6756, 训

第7180步, 一个minibatch上的损失为 0.7481, 训练准确率为 0.820
第7190步, 一个minibatch上的损失为 0.5452, 训练准确率为 0.812
第7200步, 一个minibatch上的损失为 0.6970, 训练准确率为 0.781
第7210步, 一个minibatch上的损失为 0.5236, 训练准确率为 0.820
第7220步, 一个minibatch上的损失为 0.5033, 训练准确率为 0.828
第7230步, 一个minibatch上的损失为 0.6019, 训练准确率为 0.836
第7240步, 一个minibatch上的损失为 0.5929, 训练准确率为 0.812
第7250步, 一个minibatch上的损失为 0.5251, 训练准确率为 0.844
第7260步, 一个minibatch上的损失为 0.4378, 训练准确率为 0.891
第7270步, 一个minibatch上的损失为 0.5820, 训练准确率为 0.812
第7280步, 一个minibatch上的损失为 0.5216, 训练准确率为 0.844
第7290步, 一个minibatch上的损失为 0.5981, 训练准确率为 0.828
第7300步, 一个minibatch上的损失为 0.4896, 训练准确率为 0.867
第7310步, 一个minibatch上的损失为 0.5288, 训练准确率为 0.844
第7320步, 一个minibatch上的损失为 0.5134, 训练准确率为 0.812
第7330步, 一个minibatch上的损失为 0.5965, 训练准确率为 0.812
第7340步, 一个minibatch上的损失为 0.6069, 训练准确率为 0.789
第7350步, 一个minibatch上的损失为 0.6279, 训练准确率为 0.820
第7360步, 一个minibatch上的损失为 0.5813, 训练准确率为 0.820
第7370步, 一个minibatch上的损失为 0.4574, 训练准确率为 0.836
第7380步, 一个minibatch上的损失为 0.6090, 训练准确率为 0.750
第7390步, 一个minibatch上的损失为 0.5334, 训

第8970步, 一个minibatch上的损失为 0.4314, 训练准确率为 0.836
第8980步, 一个minibatch上的损失为 0.4393, 训练准确率为 0.867
第8990步, 一个minibatch上的损失为 0.4897, 训练准确率为 0.852
第9000步, 一个minibatch上的损失为 0.4372, 训练准确率为 0.867
第9010步, 一个minibatch上的损失为 0.3137, 训练准确率为 0.922
第9020步, 一个minibatch上的损失为 0.4631, 训练准确率为 0.852
第9030步, 一个minibatch上的损失为 0.2908, 训练准确率为 0.922
第9040步, 一个minibatch上的损失为 0.4631, 训练准确率为 0.883
第9050步, 一个minibatch上的损失为 0.4633, 训练准确率为 0.852
第9060步, 一个minibatch上的损失为 0.6267, 训练准确率为 0.812
第9070步, 一个minibatch上的损失为 0.3771, 训练准确率为 0.906
第9080步, 一个minibatch上的损失为 0.5067, 训练准确率为 0.828
第9090步, 一个minibatch上的损失为 0.6741, 训练准确率为 0.836
第9100步, 一个minibatch上的损失为 0.5133, 训练准确率为 0.852
第9110步, 一个minibatch上的损失为 0.4221, 训练准确率为 0.867
第9120步, 一个minibatch上的损失为 0.4657, 训练准确率为 0.836
第9130步, 一个minibatch上的损失为 0.3906, 训练准确率为 0.867
第9140步, 一个minibatch上的损失为 0.4976, 训练准确率为 0.859
第9150步, 一个minibatch上的损失为 0.6100, 训练准确率为 0.828
第9160步, 一个minibatch上的损失为 0.4275, 训练准确率为 0.852
第9170步, 一个minibatch上的损失为 0.4083, 训练准确率为 0.875
第9180步, 一个minibatch上的损失为 0.4254, 训

### 版权归 © 稀牛学院 所有 保留所有权利
![](./graphs/xiniu_neteasy.png)